In [252]:
import pymorphy2
import string
import nltk
import numpy as np
from stop_words import get_stop_words
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem.snowball import SnowballStemmer 

In [178]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/adpashnin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/adpashnin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [232]:
with open('doc.txt', 'r') as file:
    content = file.read().replace('\n', ' ')

In [103]:
with open('another_doc.txt', 'r') as file:
    another_doc = file.read().replace('\n', ' ')

In [244]:
def tokenize_ru(file_text):
    # firstly let's apply nltk tokenization
    tokens = word_tokenize(file_text)

    # let's delete punctuation symbols
    tokens = [i for i in tokens if (i not in string.punctuation)]

    # deleting stop_words
    stop_words = stopwords.words('russian') + get_stop_words('ru')
    stop_words.extend(['в', '', '•', '—', '–', 'к', 'на', '№', '©', '►', '...'])
    tokens = [i for i in tokens if (i not in stop_words and len(i))]
    # cleaning words
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    
    morph = pymorphy2.MorphAnalyzer()
    #stems=[]
    #for item in tokens:
    #   stems.append(SnowballStemmer("russian").stem(item))
    morphs = []
    for item in tokens:
        morphs.append(sorted(morph.parse(item), key=lambda p: p.score)[0].normal_form)
    
    return morphs

In [251]:
#def tf_idf(corpus):
corpus = [content, another_doc]
stop_words = stopwords.words('russian') + get_stop_words('ru')
vectorizer = TfidfVectorizer(tokenizer = tokenize_ru)
X = vectorizer.fit_transform(corpus)
#return X

In [273]:
features = np.array(vectorizer.get_feature_names())
print(features[(-X.toarray()[0]).argsort()[:10]])

['масло' 'перец' 'нарезать' '1' 'зелень' 'соль' 'добавить' 'приготовление'
 'минуть' '2']


In [245]:
tokens = tokenize_ru(content)

In [246]:
freq_dist = nltk.FreqDist(w.lower() for w in tokens)

In [247]:
freq_dist.most_common(200)

[('масло', 5245),
 ('1', 5022),
 ('перец', 3662),
 ('соль', 3637),
 ('нарезать', 3599),
 ('2', 3165),
 ('зелень', 2904),
 ('добавить', 2566),
 ('приготовление', 2392),
 ('вымыть', 2362),
 ('л.', 2328),
 ('минуть', 2297),
 ('растительный', 2082),
 ('мука', 2064),
 ('способ', 2050),
 ('яйцо', 2046),
 ('петрушка', 2031),
 ('выложить', 2019),
 ('сливочный', 1955),
 ('мл', 1941),
 ('лука', 1923),
 ('очистить', 1902),
 ('вод', 1872),
 ('ст', 1828),
 ('ингредиент', 1799),
 ('100', 1792),
 ('ложка', 1748),
 ('длить', 1524),
 ('укроп', 1446),
 ('200', 1414),
 ('столовый', 1393),
 ('3', 1368),
 ('чеснок', 1292),
 ('перемешать', 1219),
 ('мясо', 1219),
 ('тесто', 1197),
 ('50', 1194),
 ('помидор', 1187),
 ('сахара', 1176),
 ('посолить', 1174),
 ('мелко', 1152),
 ('духовка', 1113),
 ('посыпать', 1100),
 ('', 1091),
 ('баклажан', 1077),
 ('стол', 1058),
 ('в', 1056),
 ('лист', 1052),
 ('смазать', 1046),
 ('сахар', 1043),
 ('положить', 1014),
 ('поставить', 1013),
 ('молотый', 986),
 ('готовый', 979